In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from scipy.spatial import distance as dist
from imutils import face_utils
import numpy as np
import math
import imutils
import time
import dlib
import cv2
import pandas as pd
from cv2 import VideoWriter_fourcc, VideoWriter
import matplotlib.pyplot as plt
from keras.preprocessing.image import img_to_array
from keras.models import load_model
from __future__ import print_function
import keras
from keras import layers
from keras.layers import SeparableConv2D
from keras.layers import GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense,Dropout,Activation,Flatten,BatchNormalization
from keras.layers import Conv2D,MaxPooling2D
import os
from keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from keras.preprocessing.image import img_to_array
import cv2
from keras.regularizers import l2
from keras.layers import Input
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
#задаем начальные параметры для фрейма и модели
dataset_path = '/content/drive/MyDrive/neiro_project/fer2013.csv'
image_size=(48,48)
batch_size = 32
num_epochs = 110
input_shape = (48, 48, 1)
validation_split = .2
verbose = 1
num_classes = 7
patience = 50
l2_regularization=0.01

#загружаем датасет и подготавливаем к моделе
def load_fer2013():
	data = pd.read_csv(dataset_path)
	pixels = data['pixels'].tolist()
	width, height = 48, 48
	faces = []
	for pixel_sequence in pixels:
		face = [int(pixel) for pixel in pixel_sequence.split(' ')]
		face = np.asarray(face).reshape(width, height)
		face = cv2.resize(face.astype('uint8'),image_size)
		faces.append(face.astype('float32'))
	faces = np.asarray(faces)
	faces = np.expand_dims(faces, -1)
	emotions = pd.get_dummies(data['emotion']).values
	return faces, emotions

def preprocess_input(x, v2=True):
    x = x.astype('float32')
    x = x / 255.0
    if v2:
        x = x - 0.5
        x = x * 2.0
    return x

#задаем параметры для картинки
data_generator = ImageDataGenerator(
                        featurewise_center=False,
                        featurewise_std_normalization=False,
                        rotation_range=10,
                        width_shift_range=0.1,
                        height_shift_range=0.1,
                        zoom_range=.1,
                        horizontal_flip=True)

#используем l2 регулизация
regularization = l2(l2_regularization)



In [ ]:
#задаем начальный слой модели
img_input = Input(input_shape)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(img_input)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = Conv2D(8, (3, 3), strides=(1, 1), kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)

# слой 1
residual = Conv2D(16, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(16, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# слой 2
residual = Conv2D(32, (1, 1), strides=(2, 2), padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(32, (3, 3), padding='same', kernel_regularizer=regularization, use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# слой 3
residual = Conv2D(64, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(64, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])

# слой 4
residual = Conv2D(128, (1, 1), strides=(2, 2),padding='same', use_bias=False)(x)
residual = BatchNormalization()(residual)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
x = SeparableConv2D(128, (3, 3), padding='same',kernel_regularizer=regularization,use_bias=False)(x)
x = BatchNormalization()(x)
x = MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)
x = layers.add([x, residual])
x = Conv2D(num_classes, (3, 3), padding='same')(x)
x = GlobalAveragePooling2D()(x)
output = Activation('softmax',name='predictions')(x)

#генерация самой модели с оптимизаторами, функцией ошибок и метрикой
model = Model(img_input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

#сохранение обученной модели в файл для дальнейшего использования
log_file_path = '_emotion_training.log'
csv_logger = CSVLogger(log_file_path, append=False)
early_stop = EarlyStopping('val_loss', patience=patience)
reduce_lr = ReduceLROnPlateau('val_loss', factor=0.1, patience=int(patience/4), verbose=1)
trained_models_path = 'trained_model1'
model_names = trained_models_path + '.hdf5'
model_checkpoint = ModelCheckpoint(model_names, 'val_loss', verbose=1,save_best_only=True)
callbacks = [model_checkpoint, csv_logger, early_stop, reduce_lr]

#обучение модели распознаванию
faces, emotions = load_fer2013()
faces = preprocess_input(faces)
num_samples, num_classes = emotions.shape
x_train, x_test, y_train, y_test = train_test_split(faces, emotions,test_size=0.2,shuffle=True)
model.fit_generator(data_generator.flow(x_train, y_train,
                                            batch_size),
                        steps_per_epoch=len(x_train) / batch_size,
                        epochs=num_epochs, verbose=1, callbacks=callbacks,
                        validation_data=(x_test, y_test))

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 48, 48, 1)]          0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 46, 46, 8)            72        ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 46, 46, 8)            32        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 46, 46, 8)            0         ['batch_normalization[0][0

<ipython-input-4-8a1963f561ab>:77: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(data_generator.flow(x_train, y_train,


898/897 [==============================] - ETA: 0s - loss: 1.7692 - accuracy: 0.3277
Epoch 1: val_loss improved from inf to 1.56724, saving model to trained_model1.hdf5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


897/897 [==============================] - 239s 261ms/step - loss: 1.7692 - accuracy: 0.3277 - val_loss: 1.5672 - val_accuracy: 0.4253 - lr: 0.0010
Epoch 2/110
898/897 [==============================] - ETA: 0s - loss: 1.4950 - accuracy: 0.4400
Epoch 2: val_loss improved from 1.56724 to 1.40236, saving model to trained_model1.hdf5
897/897 [==============================] - 244s 272ms/step - loss: 1.4950 - accuracy: 0.4400 - val_loss: 1.4024 - val_accuracy: 0.4791 - lr: 0.0010
Epoch 3/110
898/897 [==============================] - ETA: 0s - loss: 1.3742 - accuracy: 0.4876
Epoch 3: val_loss did not improve from 1.40236
897/897 [==============================] - 245s 273ms/step - loss: 1.3742 - accuracy: 0.4876 - val_loss: 1.4814 - val_accuracy: 0.4596 - lr: 0.0010
Epoch 4/110
898/897 [==============================] - ETA: 0s - loss: 1.3093 - accuracy: 0.5076
Epoch 4: val_loss improved from 1.40236 to 1.27317, saving model to trained_model1.hdf5
897/897 [==============================] -

In [ ]:
#считаем расстояние от глаз до бровей
def eye_brow_distance(leye,reye):
    global points
    distq = dist.euclidean(leye,reye)
    points.append(int(distq))
    return distq

#классифицируем категорию эмоций, кореллирующих со стрессом
def emotion_finder(faces,frame):
    global emotion_classifier
    EMOTIONS = ["angry" ,"disgust","fear", "happy", "sad", "surprise", "neutral"]
    x,y,w,h = face_utils.rect_to_bb(faces)
    frame = frame[y:y+h,x:x+w]
    roi = cv2.resize(frame,(48,48))
    roi = roi.astype("float") / 255.0
    roi = img_to_array(roi)
    roi = np.expand_dims(roi,axis=0)
    preds = emotion_classifier.predict(roi)[0]
    emotion_probability = np.max(preds)
    label = EMOTIONS[preds.argmax()]
    if label in ['fear', 'sad', 'neutral', 'angry']:
        label = 'stressed'
    else:
        label = 'not stressed'
    return label
#нормализуем значения расстояний от глаз до бровей и занчения сжатости губ
def normalize_values(points, disp, lip_compression):
    # Нормализация значения сжатости губ
    normalized_value_lips = abs(lip_compression - np.min(points))/abs(np.max(points) - np.min(points))
    stress_value_lips = np.exp(-(normalized_value_lips))

    # Нормализация значения расстояния между глазами и бровями
    normalized_value_eyebrow = abs(disp - np.min(points))/abs(np.max(points) - np.min(points))
    stress_value_eyebrow = np.exp(-(normalized_value_eyebrow))

    # Комбинирование значений или другие действия для получения итогового значения стресса
    combined_stress_value = (0.7 * stress_value_lips) + (0.3 * stress_value_eyebrow)

    return combined_stress_value


#подгружаем нашу модель, входное видео, а также библиотеку разметки лица
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("/content/drive/MyDrive/neiro_project/shape_predictor_68_face_landmarks.dat")
emotion_classifier = load_model("/content/drive/MyDrive/neiro_project/trained_model.hdf5", compile=True)
print(emotion_classifier, flush = True)
cap = cv2.VideoCapture('/content/drive/MyDrive/neiro_project/in4.MOV')

#дальше видео с лицом разбивается на фреймы, и каждый фрейм анализируется на наличие стресса
points = []
stress_list = []
stressval_list = []
stressgraph = []
size=0
tmp_strtess_value = []
while(True):
    _,frame = cap.read()
    if(not _): break
    frame = cv2.flip(frame,1)
    frame = imutils.resize(frame, width=500,height=500)


    (lBegin, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eyebrow"]
    (rBegin, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eyebrow"]

    gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

    detections = detector(gray,0)
    for detection in detections:
        emotion = emotion_finder(detection, gray)
        shape = predictor(gray, detection)
        shape = face_utils.shape_to_np(shape)

        lip_left_corner = (shape[48][0], shape[48][1])
        lip_right_corner = (shape[54][0], shape[54][1])
        lip_top_center = (shape[51][0], shape[51][1])
        lip_bottom_center = (shape[57][0], shape[57][1])

        cv2.line(frame, lip_left_corner, lip_right_corner, (0, 255, 0), 2)
        cv2.line(frame, lip_top_center, lip_bottom_center, (0, 255, 0), 2)

        lip_compression = abs(lip_top_center[1] - lip_bottom_center[1])

        #cv2.putText(frame, f"Lip Compression: {lip_compression}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
        cv2.putText(frame, emotion, (10,10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)


        leyebrow = shape[lBegin:lEnd]
        reyebrow = shape[rBegin:rEnd]

        reyebrowhull = cv2.convexHull(reyebrow)
        leyebrowhull = cv2.convexHull(leyebrow)

        cv2.drawContours(frame, [reyebrowhull], -1, (0, 0, 255), 1)
        cv2.drawContours(frame, [leyebrowhull], -1, (0, 0, 255), 1)

        distq = eye_brow_distance(leyebrow[-1],reyebrow[0])
        stress_value = normalize_values(points, distq, lip_compression)
        tmp_strtess_value.append(stress_value)
        print(stress_value)
        if math.isnan(stress_value):
            continue
        cv2.putText(frame,"stress level:{}".format(str(int(stress_value*100))),(20,40),cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 1)
        stress_list.append(frame)
    height, width, layers = frame.shape
    size = (width,height)
    stressval_list.append(stress_value)
out = cv2.VideoWriter('output_video.avi',cv2.VideoWriter_fourcc(*'DIVX'), 10, size)
cap.release()
print("END OF TEST")
print(tmp_strtess_value)
print(np.array(tmp_strtess_value).mean())
for i in range(len(stress_list)):
    out.write(stress_list[i])


In [ ]:
#финальный вердикт считается на основе выходных данных модели и порога
def log_reg(tmp_strtess_value):
  threshold = 0.5
  y = np.array([1 if value > threshold else 0 for value in tmp_strtess_value])
  x = np.array(tmp_strtess_value).reshape(-1, 1)
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
  logistic_regression = LogisticRegression(max_iter=1000)
  logistic_regression.fit(x_train, np.ravel(y_train))
  predictions = logistic_regression.predict(x_test)
  count_ones = sum(predictions)
  count_zeros = len(predictions) - count_ones
  if count_ones > count_zeros:
    print("У человека вероятно стресс")
  elif count_ones < count_zeros:
    print("У человека вероятно нет стресса")
  else:
    print("Неопределенность")
  accuracy = accuracy_score(y_test, predictions)
log_reg(tmp_strtess_value)

У человека вероятно стресс
